#https://machinelearningmastery.com/how-to-use-an-encoder-decoder-lstm-to-echo-sequences-of-random-integers/

In [1]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

Using TensorFlow backend.


In [2]:
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y

In [3]:
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y


In [4]:
# define LSTM
n_in = 5
n_out = 2
encoded_length = 100
batch_size = 21
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
model.add(RepeatVector(n_out))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

Epoch 1/1
 - 3s - loss: 4.6039 - acc: 0.0238
Epoch 1/1
 - 0s - loss: 4.6040 - acc: 0.0238
Epoch 1/1
 - 0s - loss: 4.6046 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6057 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6042 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.6033 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6038 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6046 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.6050 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6043 - acc: 0.0238
Epoch 1/1
 - 0s - loss: 4.6008 - acc: 0.0238
Epoch 1/1
 - 0s - loss: 4.6048 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.5999 - acc: 0.0714
Epoch 1/1
 - 0s - loss: 4.6029 - acc: 0.0238
Epoch 1/1
 - 0s - loss: 4.6084 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6023 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6030 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6008 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.6034 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.6022 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.5986 - acc: 0.0000e+00
Epoch 1/1
 

Epoch 1/1
 - 0s - loss: 3.4283 - acc: 0.3095
Epoch 1/1
 - 0s - loss: 3.8619 - acc: 0.1429
Epoch 1/1
 - 0s - loss: 3.9589 - acc: 0.2143
Epoch 1/1
 - 0s - loss: 3.9100 - acc: 0.1429
Epoch 1/1
 - 0s - loss: 3.6024 - acc: 0.2143
Epoch 1/1
 - 0s - loss: 3.6329 - acc: 0.2143
Epoch 1/1
 - 0s - loss: 3.6257 - acc: 0.2381
Epoch 1/1
 - 0s - loss: 3.3945 - acc: 0.2619
Epoch 1/1
 - 0s - loss: 3.9733 - acc: 0.1429
Epoch 1/1
 - 0s - loss: 3.6759 - acc: 0.1667
Epoch 1/1
 - 0s - loss: 3.4644 - acc: 0.2857
Epoch 1/1
 - 0s - loss: 3.8593 - acc: 0.0952
Epoch 1/1
 - 0s - loss: 3.4310 - acc: 0.3333
Epoch 1/1
 - 0s - loss: 3.3652 - acc: 0.2381
Epoch 1/1
 - 0s - loss: 3.5070 - acc: 0.2143
Epoch 1/1
 - 0s - loss: 3.4327 - acc: 0.2143
Epoch 1/1
 - 0s - loss: 3.8858 - acc: 0.2857
Epoch 1/1
 - 0s - loss: 3.8426 - acc: 0.1667
Epoch 1/1
 - 0s - loss: 3.6123 - acc: 0.3333
Epoch 1/1
 - 0s - loss: 3.8594 - acc: 0.1667
Epoch 1/1
 - 0s - loss: 3.6979 - acc: 0.1667
Epoch 1/1
 - 0s - loss: 3.4344 - acc: 0.2857
Epoch 1/1


Epoch 1/1
 - 0s - loss: 3.1479 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.9882 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 2.7491 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 2.9405 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.3512 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 3.0439 - acc: 0.3095
Epoch 1/1
 - 0s - loss: 3.2215 - acc: 0.3333
Epoch 1/1
 - 0s - loss: 2.8807 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.8840 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 2.8157 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.8387 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 3.3053 - acc: 0.3095
Epoch 1/1
 - 0s - loss: 2.9782 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 2.7906 - acc: 0.3333
Epoch 1/1
 - 0s - loss: 3.1436 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 3.0575 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 3.2528 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.7916 - acc: 0.4286
Epoch 1/1
 - 0s - loss: 2.6856 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 3.0450 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 3.2853 - acc: 0.3810
Epoch 1/1
 - 0s - loss: 2.7342 - acc: 0.4762
Epoch 1/1


 - 0s - loss: 2.4072 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 2.4380 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 2.4927 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.2913 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.7254 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.3351 - acc: 0.4286
Epoch 1/1
 - 0s - loss: 2.4303 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.7730 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 2.6991 - acc: 0.4286
Epoch 1/1
 - 0s - loss: 2.8057 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.3419 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.5499 - acc: 0.4048
Epoch 1/1
 - 0s - loss: 2.5175 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.6489 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.4082 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.5948 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.4249 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.4772 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.3923 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.5944 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.6571 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.6741 - acc: 0.5000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 2.4912 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.2728 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.3812 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.1209 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.7045 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.3666 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.5393 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.6374 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.5320 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.3163 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.2951 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.8378 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.3627 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 2.5808 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.5056 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.5559 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.4865 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.8071 - acc: 0.4762
Epoch 1/1
 - 0s - loss: 2.1810 - acc: 0.5000
Epoch 1/1
 - 0s - loss: 2.3998 - acc: 0.4524
Epoch 1/1
 - 0s - loss: 2.1129 - acc: 0.5238
Epoch 1/1
 - 0s - loss: 2.4332 - acc: 0.5000
Epoch 1/1


...

Epoch 1/1
 - 0s - loss: 0.6991 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.4115 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5166 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5455 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.3981 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.5705 - acc: 0.7857
Epoch 1/1
 - 0s - loss: 0.5430 - acc: 0.7857
Epoch 1/1
 - 0s - loss: 0.5126 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.4845 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5627 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5750 - acc: 0.8095
Epoch 1/1
 - 0s - loss: 0.6547 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.6874 - acc: 0.7619
Epoch 1/1
 - 0s - loss: 0.5770 - acc: 0.7857
Epoch 1/1
 - 0s - loss: 0.4643 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.5538 - acc: 0.8095
Epoch 1/1
 - 0s - loss: 0.5048 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.4474 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.6428 - acc: 0.8095
Epoch 1/1
 - 0s - loss: 0.4412 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.4808 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.5951 - acc: 0.8095
Epoch 1/1


 - 0s - loss: 0.4736 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3800 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4858 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3219 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.5363 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.4721 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3973 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5454 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3758 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4327 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.5373 - acc: 0.8333
Epoch 1/1
 - 0s - loss: 0.5422 - acc: 0.7857
Epoch 1/1
 - 0s - loss: 0.3202 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.4755 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.4296 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4691 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4497 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.3988 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3806 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3479 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3334 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3204 - acc: 0.9286
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.3747 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3579 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3697 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.4958 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4248 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3215 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3810 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.4273 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3200 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.3225 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3520 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3252 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3727 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.4976 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.4223 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3824 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.4032 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3119 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3250 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.3177 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.4462 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.3936 - acc: 0.8333
Epoch 1/1


Epoch 1/1
 - 0s - loss: 0.3869 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3329 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2975 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3912 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3737 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3051 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.2361 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.3821 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3381 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.2396 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3827 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3622 - acc: 0.8571
Epoch 1/1
 - 0s - loss: 0.4447 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3142 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3687 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.3366 - acc: 0.8810
Epoch 1/1
 - 0s - loss: 0.2909 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3428 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.5343 - acc: 0.8095
Epoch 1/1
 - 0s - loss: 0.3548 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2769 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 0.3266 - acc: 0.9286
Epoch 1/1


Epoch 1/1
 - 0s - loss: 0.1468 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2600 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2699 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.2255 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2653 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.1822 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2386 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2211 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2913 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.2684 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2448 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2518 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2920 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.2597 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3897 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2677 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.2077 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3590 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.2629 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 0.3240 - acc: 0.9762
Epoch 1/1
 - 0s - loss: 0.2865 - acc: 0.9286
Epoch 1/1
 - 0s - loss: 0.3685 - acc: 0.9048
Epoch 1/1
